## We will create a dashboard that can accept user input.

Below are all features in this dataset

- Title: Title of product sold
- TITLE_CODE: Unique ID for titles
- CHANNEL: Masked channel description of channel through which the product was distributed to the customer
- PROD_TYP: Indicator if the product is a paperback or hardback
- SERIES: Y/N indicator if the product is part of a series
- CH1_GENRE: Genre listing for product from Channel 1 database
- CH1_THEME: Theme listing for product from Channel 1 database
- CH2_CATEGORY: Category listing for product from Channel 2 database
- CH2_SUBCATEGORY: Subcategory listing for product from Channel 2 database
- LEXILE_11_DESC: Lexile measures for product. Note this field is not always complete for every Scholastic product. For more information on Lexile codes, see links below:
 - : https://lexile.com/educators/measuring-growth-with-lexile/lexile-measures-grade-equivalents/
 - : https://lexile.com/educators/find-books-at-the-right-level/about-lexile-codes/
- total_units: Number of products sold 
- UNIT_PRICE: Unit price of product sold
- SCHOOL_TYPE: Indicator if the school where the product was sold was public or not.
- REGION: Region of the United States where product was distributed - NORTHEAST, MIDWEST, SOUTH, WEST, or OTHER
- STATE: US state of sale
- COUNTY: County of sale
- EDU_NO_HS: % of population with no HS degree, by zip code
- EDU_HS_SOME_COLLEGE: % of population with some college, by zip code
- EDU_BACHELOR_DEG: % of population with bachelor degree, by zip code
- EDU_GRADUATE_DEG: % of population with graduate degree, by zip code
- HHI_BAND: Bands of household income for zip code, in $10,000 bands
- ZIP_CODE: Zip code of sale

In this exercise, we will create a dashboard to show books sales by channel

####Load the dataset

In [0]:
scholastic=spark.read.option("delimiter", "|").csv("/FileStore/tables/scholastic/sdata.csv", header=True, inferSchema=True)

scholastic.createOrReplaceTempView("BooksTable")

### Notice about widgets

Input widgets allow you to add parameters to your notebooks and dashboards. The widget API consists of calls to create various types of input widgets, remove them, and get bound values.

### Widgets are best for:
- Building a notebook or dashboard that is re-executed with different parameters
- Quickly exploring results of a single query with different parameters
##### Widgets documentation can be found [here](https://docs.databricks.com/notebooks/widgets.html)
##### Visulazation documentation can be found [here](https://docs.databricks.com/notebooks/visualizations/index.html)

In [0]:
# Delete all widgets
dbutils.widgets.removeAll()

Create a dropdown widget for channel

In [0]:
%sql
CREATE WIDGET DROPDOWN Channel DEFAULT "CHANNEL 1" 
CHOICES 
SELECT DISTINCT channel 
FROM BooksTable
ORDER BY channel

Create a multiselect widget for channel

In [0]:
%sql
CREATE WIDGET multiselect Channel_multi DEFAULT "CHANNEL 1" 
CHOICES 
SELECT DISTINCT channel 
FROM BooksTable
ORDER BY channel

### Calculate total sales by product type based on the channel of a user's choice (single choice)

In [0]:
%sql
select prod_typ, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where channel=getArgument("Channel")
group by prod_typ

prod_typ,total_sales
PAPERBACK,9948612.24
HARDBACK,1055352.54


### Calculate total sales by product type based on the channel of a user's choice (multiple choice)

In [0]:
%sql
select prod_typ, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where channel = split(getArgument("Channel_multi"),",")[0] or channel = split(getArgument("Channel_multi"),",")[1]
group by prod_typ

prod_typ,total_sales
PAPERBACK,9948612.24
HARDBACK,1055352.54


### Number of sales by state based on the channel of a user's choice (single choice)

In [0]:
%sql

select state, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where channel = getArgument("Channel")
group by state


state,total_sales
K,89.69
NJ,218600.4
null,60560.71
RI,31234.64
NH,54453.43
CT,95446.77
VT,24465.74
VI,3015.93
ME,80126.33
AE,6450.48


### Number of sales by school type based on the channel of a user's choice (single choice)

In [0]:
%sql
select school_type, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where channel = getArgument("Channel")
group by school_type

school_type,total_sales
PUBLIC,8692461.31
OTHER,2311503.47


Number of sales by series based on the channel of a user's choice (single choice)

In [0]:
%sql
select series, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where channel = getArgument("Channel")
group by series

series,total_sales
Y,1820414.28
N,5713565.78


## Book Search

Create a widget to find all books where title includes a word enter by user for specific channel

In [0]:
# remove titel widget if exist

dbutils.widgets.remove("title")

In [0]:
%sql

CREATE WIDGET TEXT Book_Title DEFAULT "" 

In [0]:
%sql

select title, round(sum(total_units*unit_price),2) as total_sales
from bookstable
where instr(lower(title), lower(getArgument("Book_Title")))>=1 and channel = getArgument("Channel")
group by title
order by total_sales desc


title,total_sales
Dog Man: Lord of the Fleas,359170.47
Super Deluxe Essential Handbook,102516.61
Kristy's Big Day,98866.04
"I Survived the Attack of the Grizzlies, 1967",79745.19
"Collector, The",72814.83
Roblox: The Essential Guide,69340.62
Dog Man and Cat Kid,66373.56
KleptoCats: It's Their World Now!,64703.02
"Three Little Superpigs, The",61786.85
Resistance,58966.2
